In [1]:

import numpy as np
from PIL import Image  # Import PIL for image processing
from numba import njit, prange
from matplotlib import pyplot as plt
import os
import random
from tqdm import tqdm


In [2]:

###variable declarations

s = 200                          # Size of the square domain in µm
n = 800                          # Number of grid points

sigma = 0.25                     # Simulation constant (D*dt / dx^2)

D0_free = 1.5e-5                 # Diffusion coefficient free in cm^2/s
D0_cell = 1.5e-5                 # Diffusion coefficient in cell in cm^2/s
D0_wall = 1.5e-7                 # Diffusion coefficient throug cell membrane in cm^2/s

image_path = "Cell .png"         # path to import cell image

molecule_number = 1000           # Number of released DA molecules per release
release_freq = 1                 # DA release frequency in Hz
num_release_positions = 100      # Number of release positions

k_on = 1e6                       # Rate constant of the binding reaction in M^(-1)*s^(-1)
k_off = 0.5                      # Rate constant of the unbinding reaction in s^(-1)

sensor_density = 4.0             # Sensors per square micrometer
binding_sites = 30               # Number of binding sites per sensor

dt_output = 500                  # Time interval for data output in milliseconds

th = 10                          # half life of ROS in s


In [3]:

###pre-calculations

dx = s / (n - 1)                           # Step size in µm
dy = dx      
D_free = D0_free * 1e5 * (1/dx)**2         # Convert from cm^2/s to (grid size)^2/ms
D_cell = D0_cell * 1e5 * (1/dx)**2         # Convert from cm^2/s to (grid size)^2/ms
D_wall = D0_wall * 1e5 * (1/dx)**2         # Convert from cm^2/s to (grid size)^2/ms
ts = sigma / D_free                        # Time step (in ms)
release_time = 1000 / release_freq         # Time between release events (in ms)

pixel_area = dx * dy                       # Area of a single pixel in square micrometers
num_bind = sensor_density * binding_sites * pixel_area  # Number of binding sites per pixel

er = np.exp(-ts / (th * 1000))             # Factor for Decay due to half-life 


In [4]:

###Define Areas in- and outside the cell
def area_matrix(image_path, n):
    """Generate the area matrix based on an imported image."""
    
    # Load the image
    image = Image.open(image_path)

    # Convert the image to grayscale
    image = image.convert("L")

    # Resize the image to match the simulation grid
    image = image.resize((n, n))

    # Convert the image to a numpy array
    image_array = np.array(image)

    # Normalize the image array to binary (0 and 1) based on pixel intensity
    threshold = 128  # You can adjust this threshold if needed
    binary_array = np.where(image_array < threshold, 1, 0)  # 1 for black, 0 for white

    return binary_array


In [5]:

###Define diffucion matrices
def diffusion_matrices(D_cell, D_free, D_wall, area):
    D_x = np.zeros((n, n))                       
    D_neg_x = np.zeros((n, n))                   
    D_y = np.zeros((n, n))                       
    D_neg_y = np.zeros((n, n))                  
    
    for i in range (1, n-1):
        for j in range (1, n-1):
            
            if area[i,j] == 1 and area[i, j-1] == 1:
                D_x[i,j] = D_cell
            elif area[i,j] == 0 and area[i, j-1] == 0:
                D_x[i,j] = D_free
            else:
                D_x[i,j] = D_wall
                
            if area[i,j] == 1 and area[i, j+1] == 1:
                D_neg_x[i,j] = D_cell
            elif area[i,j] == 0 and area[i, j+1] == 0:
                D_neg_x[i,j] = D_free
            else:
                D_neg_x[i,j] = D_wall
                
            if area[i,j] == 1 and area[i-1, j] == 1:
                D_y[i,j] = D_cell
            elif area[i,j] == 0 and area[i-1, j] == 0:
                D_y[i,j] = D_free
            else:
                D_y[i,j] = D_wall
                
            if area[i,j] == 1 and area[i+1, j] == 1:
                D_neg_y[i,j] = D_cell
            elif area[i,j] == 0 and area[i+1, j] == 0:
                D_neg_y[i,j] = D_free
            else:
                D_neg_y[i,j] = D_wall    
                
    return(D_x, D_neg_x, D_y, D_neg_y)


In [6]:

### Function to generate release positions randomly at borders of the cell
def generate_release_positions(area, num_release_positions):
    
    n = area.shape[0]
    border_positions = []
    inner_positions = []

    # Identify border and inner positions
    for i in range(1, n-1):
        for j in range(1, n-1):
            if area[i, j] == 0:  
                if (area[i-1, j] == 1 or area[i+1, j] == 1 or
                    area[i, j-1] == 1 or area[i, j+1] == 1):
                    border_positions.append((i, j))
            elif area[i, j] == 1:  # Inside cell
                inner_positions.append((i, j))

    num_border_positions = 0
    num_inner_positions = num_release_positions - num_border_positions

    selected_border_positions = random.sample(border_positions, min(num_border_positions, len(border_positions)))
    selected_inner_positions = random.sample(inner_positions, min(num_inner_positions, len(inner_positions)))

    release_positions = selected_border_positions + selected_inner_positions
    return release_positions



In [7]:

### Create output directories if they don't exist

os.makedirs("output_txt", exist_ok=True)
os.makedirs("output_images", exist_ok=True)
os.makedirs("output_bound_images", exist_ok=True)


In [8]:

# Runge-Kutta method for diffusion with Numba parallelization
@njit(parallel=True)
def diffuse(u, D_x, D_neg_x, D_y, D_neg_y):
    du = np.zeros_like(u)

    du[1:-1, 1:-1] = (D_y[1:-1, 1:-1]  * u[:-2, 1:-1] + 
                      D_neg_y[1:-1, 1:-1]  * u[2:, 1:-1] + 
                      D_x[1:-1, 1:-1] * u[1:-1, :-2] + 
                      D_neg_x[1:-1, 1:-1]  * u[1:-1, 2:] - 
                      (D_y[1:-1, 1:-1]+D_neg_y[1:-1, 1:-1]+D_x[1:-1, 1:-1]+D_neg_x[1:-1, 1:-1]) * u[1:-1, 1:-1])
    return du

@njit(parallel=True)
def runge_kutta(u, D_x, D_neg_x, D_y, D_neg_y):
    k1 = diffuse(u, D_x, D_neg_x, D_y, D_neg_y)
    k2 = diffuse(u + 0.5 * ts * k1, D_x, D_neg_x, D_y, D_neg_y)
    k3 = diffuse(u + 0.5 * ts * k2, D_x, D_neg_x, D_y, D_neg_y)
    k4 = diffuse(u + ts * k3, D_x, D_neg_x, D_y, D_neg_y)
    return u + (ts / 6) * (k1 + 2*k2 + 2*k3 + k4)
          
# Take into account half-life of molecules in cell                          
@njit(parallel=True)
def decay(u, area,er):
    for i in prange(n):
        for j in prange(n):
            if area[i, j] == 1:  
                u[i, j] *= er
    return u

def wash (u, area, step, t, ts):
    if step == int(10/14 * t/ts):    
        for i in prange(n):
            for j in prange(n):
                if area[i, j] == 0:  
                    u[i, j] = 0
    return u

    
def release(u, step, t, de, release_positions, release_time, ts):
    if step * ts <= 10/14 * t: 
        if step % int(release_time / ts) == 0:  
            for (x,y) in release_positions:
                u[x, y] += de  
            
# Function for chemical binding
@njit(parallel=True)
def bind(u, b, step):
    step_number = int(10/ts)                              
    if step % step_number == 0:
        bound_molecules = np.copy(b)                     
        for i in prange(n):
            for j in prange(n):
                num_unbound = num_bind - b[i, j]          
                for _ in range(int(num_unbound)):
                    if np.random.rand() < (k_on/1000) * ts * step_number * u[i,j] * 1.66e-9:  
                        bound_molecules[i, j] += 1
                        u[i, j] -= 1
                num_bound = b[i, j]                      
                for _ in range(int(num_bound)):
                    if np.random.rand() < (k_off/1000) * ts * step_number: 
                        bound_molecules[i, j] -= 1
                        u[i, j] += 1
        return bound_molecules
    else:
        return b
    

In [9]:

# Main simulation loop with data export and concentration increase
def simulate(t, dt_output):
    
    u = np.zeros((n, n))  
    b = np.zeros((n, n))
    
    num_steps = int(t / ts)
    output_steps = int(dt_output / ts)
    
    data_array = np.zeros((num_steps // output_steps + 1, 3))  
    idx = 0  # Index for data_array
    
    area = area_matrix(image_path, n)                  
    
    D_x, D_neg_x, D_y, D_neg_y = diffusion_matrices(D_cell, D_free, D_wall, area)
    
    release_positions = generate_release_positions(area, num_release_positions) 
  
    for step in tqdm(range(num_steps)):
        release(u, step, t, molecule_number, release_positions, release_time, ts)
        u = runge_kutta(u, D_x, D_neg_x, D_y, D_neg_y)
        u = decay(u, area,er)
        u = wash (u, area, step, t, ts)
        b = bind(u, b, step)  
        if step % output_steps == 0:
            
            # Save concentration as txt file
            np.savetxt(f'output_txt/concentration_{step // output_steps:04d}.txt', u)
            # Save bound molecules as txt file
            np.savetxt(f'output_txt/bound_molecules_{step // output_steps:04d}.txt', b)
              
    # Save data array as txt file
    np.savetxt('output_txt/data_array.txt', data_array, header='Time (ms)\tTotal Molecules\tTotal Bound Molecules', fmt='%.6f', delimiter='\t')


In [10]:

# Run simulation
final_data = simulate(140000, dt_output)  # Run for 12.5 ms with output every 0.2s


100%|████████████████████████████████████████████████████████████████████████| 205821/205821 [02:40<00:00, 1282.38it/s]


In [11]:
# Data Handling and plotting

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Use the Agg backend for headless environments
plt.switch_backend('Agg')


def process_file(file_name):
    fig, ax = plt.subplots()

    try:
        file_path = os.path.join("output_txt", file_name)
        time_step = int(file_name.split("_")[-1].split(".")[0])

        if file_name.startswith("concentration_"):
            u = np.loadtxt(file_path)
            if u.size > 0:  # Ensure that data is not empty
                img = ax.imshow(u, cmap='viridis', extent=(-s/2, s/2, -s/2, s/2), vmin=0, vmax=200)
                plt.colorbar(img, ax=ax, label='DA concentration [nM]')
                ax.set_title(f'Total Molecules at t = {int(time_step * dt_output):06d} ms')
                ax.set_xlabel('x [µm]')
                ax.set_ylabel('y [µm]')
                plt.savefig(os.path.join("output_images", f"2D_total_{time_step:04d}.png"), dpi=128)
                ax.clear()

        elif file_name.startswith("bound_molecules_"):
            b = np.loadtxt(file_path)
            if b.size > 0:  # Ensure that data is not empty
                img = ax.imshow(b, cmap='viridis_r', extent=(-s/2, s/2, -s/2, s/2), vmin=0, vmax=100)
                plt.colorbar(img, ax=ax, label='Bound Molecules per Sensor')
                ax.set_title(f'Bound Molecules at t = {int(time_step * dt_output):06d} ms')
                ax.set_xlabel('x [µm]')
                ax.set_ylabel('y [µm]')
                plt.savefig(os.path.join("output_bound_images", f"2D_bound_{time_step:04d}.png"), dpi=128)
                ax.clear()

    except Exception as e:
        print(f"Error processing file: {file_name} - {e}")

    plt.close(fig)
    return True

def generate_images_from_txt():
    files = sorted(os.listdir("output_txt"))  # Ensure files are processed in order

    # Process every 5th file
    files_to_process = files[::20]

    for file_name in tqdm(files_to_process):
        process_file(file_name)


In [12]:

# Generate images from <text files
generate_images_from_txt()

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:06<00:00,  4.79it/s]


In [13]:
def define_and_visualize_manual_rois(area):
    """Manually define ROIs and visualize them."""
    n = area.shape[0]

    # Manually defined ROIs
    roi1_start = (52, 41)  # Start pixel for ROI1
    roi2_start = (52, 33)  # Start pixel for ROI2
    roi3_start = (52, 28)  # Start pixel for ROI3
    roi_size = 5  # ROIs are 10x10

    roi1 = [(i, j) for i in range(roi1_start[0], roi1_start[0] + roi_size)
                    for j in range(roi1_start[1], roi1_start[1] + roi_size)]
    roi2 = [(i, j) for i in range(roi2_start[0], roi2_start[0] + roi_size)
                    for j in range(roi2_start[1], roi2_start[1] + roi_size)]
    roi3 = [(i, j) for i in range(roi3_start[0], roi3_start[0] + roi_size)
                    for j in range(roi3_start[1], roi3_start[1] + roi_size)]

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(area, cmap='gray', extent=(0, n, n, 0))

    for (i, j) in roi1:
        ax.add_patch(plt.Rectangle((j, i), 1, 1, fill=False, edgecolor='red', lw=1))
    for (i, j) in roi2:
        ax.add_patch(plt.Rectangle((j, i), 1, 1, fill=False, edgecolor='blue', lw=1))
    for (i, j) in roi3:
        ax.add_patch(plt.Rectangle((j, i), 1, 1, fill=False, edgecolor='green', lw=1))

    ax.set_xticks([])
    ax.set_yticks([])
    plt.savefig("manual_roi_visualization.png")  # Save the plot
    print("Manual ROI visualization saved as 'manual_roi_visualization.png'.")

    return roi1, roi2, roi3


def analyze_rois(bound_data_path, roi1, roi2, roi3, output_file="roi_analysis.txt"):
    """Analyze bound molecules in the specified ROIs."""
    # Load bound molecules data from simulation
    bound_files = sorted([f for f in os.listdir(bound_data_path) if f.startswith("bound_molecules_")])
    
    roi1_totals = []
    roi2_totals = []
    roi3_totals = []

    for file in bound_files:
        bound_matrix = np.loadtxt(os.path.join(bound_data_path, file))
        roi1_total = sum(bound_matrix[i, j] for i, j in roi1)
        roi2_total = sum(bound_matrix[i, j] for i, j in roi2)
        roi3_total = sum(bound_matrix[i, j] for i, j in roi3)

        roi1_totals.append(roi1_total)
        roi2_totals.append(roi2_total)
        roi3_totals.append(roi3_total)

    # Save analysis results to a file
    with open(output_file, "w") as f:
        f.write("Time Step\tInside Total\tBorder Total\tOutside Total\n")
        for step, (r1, r2, r3) in enumerate(zip(roi1_totals, roi2_totals, roi3_totals)):
            f.write(f"{step}\t{r1}\t{r2}\t{r3}\n")

    print(f"ROI analysis saved as '{output_file}'.")


area = area_matrix(image_path, n)  # Use the imported image to generate the area
roi1, roi2, roi3 = define_and_visualize_manual_rois(area)  # Define and visualize ROIs
analyze_rois("output_txt", roi1, roi2, roi3)  # Analyze and save bound molecules data



Manual ROI visualization saved as 'manual_roi_visualization.png'.
ROI analysis saved as 'roi_analysis.txt'.
